# Dependencies

In [ ]:
from spmf import Spmf
from pathlib import Path
import logging as lg
import os
import time
import matplotlib.pyplot as plt

In [ ]:
lg.basicConfig(level=lg.INFO, force=True)
logger = lg.getLogger(__name__)

# Parameters
possible:
['test.txt', 'output', '.ipynb_checkpoints', 'kosorak.txt', 'fifa.txt', 'e_shop.txt', 'BMS1.txt', 'sign.txt', 'microblogPCU.txt']


In [ ]:
possible_names = ['test', 'kosorak', 'fifa', 'e_shop', 'BMS1', 'sign', 'microblogPCU']
input_sequence_db_name = "e_shop"
input_sequence_db_name = input_sequence_db_name + ".txt"

min_support = 220
max_seq = 1000

## Prepare Paths

In [ ]:
try:
    base_path = Path("data").resolve(True)
except Exception:
    raise Exception("You don't have data folder in this directory")
possible_sequences_names = [name.name for name in base_path.iterdir()]
input_sequence_filepath = (base_path / input_sequence_db_name).resolve(True)
output_base = base_path / "output"
output_base.mkdir(exist_ok=True)
output_filepath = output_base / input_sequence_db_name
spmf_output_filepath = output_base / f"spmf_{input_sequence_db_name}"

# Plots

In [ ]:
def create_time_plot(sups: list, calc_times: list):
    plt.figure(figsize=(8, 6))
    plt.plot(sups, calc_times, marker='o')
    plt.title("Czas wyznaczenia sekwencji spełniających minimalne wsparcie")
    plt.xlabel("Minimalne wsparcie")
    plt.ylabel("Czas Obliczeń [s]")
    plt.show()

In [ ]:
def create_freq_seq_count_plot(sups: list, seq_counts: list):
    plt.figure(figsize=(8, 6))
    plt.plot(sups, seq_counts, marker='o')
    plt.title("Ilość sekwencji spełniających minimalne wsparcie")
    plt.xlabel("Minimalne wsparcie")
    plt.ylabel("Liczba sekwencji")
    plt.show()

# Algorithm

## utils

In [ ]:
def calculate_percentage_min_support(min_support: int, input_filepath: Path) -> float:
    seq_length = 0
    with open(input_filepath, 'r') as file:
        for line in file:
            seq_length += 1
    return min_support / seq_length if seq_length > 0 else 0

In [ ]:
def reduce_size(new_seq_size: int, input_filepath: Path) -> Path:
    with open(input_filepath, 'r') as file:
        sequences = []
        for line in file:
            sequences.append(line)
    new_file = input_filepath.parent / f"{new_seq_size}_{input_filepath.name}"
    with open(new_file, "w") as file:
        file.writelines(sequences[:new_seq_size])
    return new_file

In [ ]:
class PrefixSpan:
    def __init__(self, input_filepath: Path, output_filepath: Path, min_support: int):
        self.input_filepath = input_filepath
        self.output_filepath = output_filepath
        self.min_support = min_support
        self.sequences = self.read_data(input_filepath.as_posix())
        self.frequent_patterns = []

    @staticmethod
    def read_data(filename: str):
        with open(filename, 'r') as file:
            sequences = []
            for line in file:
                sequence = [item.split() for item in line.strip().split('-1')[:-1]]
                sequences.append(sequence)
            return sequences[:max_seq]

    def prefix_span(self, prefix, projected_db):
        # Count all items and their supports in the projected_db
        item_counts = {}
        for sequence in projected_db:
            found_items = set()
            for itemset in sequence:
                for item in itemset:
                    if item not in found_items:
                        if item in item_counts:
                            item_counts[item] += 1
                        else:
                            item_counts[item] = 1
                        found_items.add(item)

        # Filter items by minimum support and recursively explore extensions
        frequent_items = [(item, count) for item, count in item_counts.items() if count >= self.min_support]
        for item, _ in sorted(frequent_items, key=lambda x: x[1], reverse=True):
            new_prefix = prefix + [item]
            self.frequent_patterns.append((new_prefix, item_counts[item]))
            suffix_db = self._build_suffix_db(new_prefix, projected_db)
            self.prefix_span(new_prefix, suffix_db)

    def _build_suffix_db(self, prefix, projected_db):
        suffix_db = []
        for sequence in projected_db:
            for index, itemset in enumerate(sequence):
                if prefix[-1] in itemset:
                    suffix = []
                    for future_index in range(index + 1, len(sequence)):
                        suffix.append(sequence[future_index])
                    if suffix:
                        suffix_db.append(suffix)
                    break
        return suffix_db

    def run(self):
        initial_db = [seq for seq in self.sequences]
        self.prefix_span([], initial_db)
        return self.frequent_patterns

    def write_frequent_patterns_to_file(self):
        if self.frequent_patterns:
          with open(self.output_filepath.as_posix(), 'w') as file:
              for pattern, support in self.frequent_patterns:
                  file.write(' '.join(pattern) + ' -1 #SUP: ' + str(support) + '\n')
        else:
            logger.warning("Trying to save output file if there is no frequent found, probably algorithm was not run")

# Experiments

In [ ]:
input_sequence_filepath = reduce_size(max_seq, input_sequence_filepath)

## SPMF version

In [ ]:
min_support_perc = calculate_percentage_min_support(min_support, input_sequence_filepath)

In [ ]:
spmf_prefix_span = Spmf("PrefixSpan", input_filename=input_sequence_filepath.as_posix(),
            output_filename=spmf_output_filepath.as_posix(), arguments=[min_support_perc])
start_time = time.perf_counter()
spmf_prefix_span.run()
end_time = time.perf_counter()
print(f"Time of smpf prefix time: {end_time - start_time} with min support: {min_support}")

## Custom implementation

In [ ]:
prefix_span = PrefixSpan(input_sequence_filepath, output_filepath, min_support)
start_time = time.perf_counter()
freq_patters = prefix_span.run()
end_time = time.perf_counter()
print(f"Time of custom prefix time: {end_time - start_time} with min support: {min_support}, found: {len(freq_patters)} sequences")
prefix_span.write_frequent_patterns_to_file()

In [ ]:
times = []
supps = []
freq_counts = []
for min_sup in range(min_support, 500, 2):
    prefix_span = PrefixSpan(input_sequence_filepath, output_filepath, min_sup)
    start_time = time.perf_counter()
    freq_patters = prefix_span.run()
    end_time = time.perf_counter()
    print(min_sup, "DONE")
    supps.append(min_sup)
    freq_counts.append(len(freq_patters))
    times.append(end_time-start_time)

# Analyse

In [ ]:
create_freq_seq_count_plot(supps, freq_counts)

In [ ]:
create_time_plot(supps, times)